In [ ]:
import pandas as pd
import csv
import datetime as dt
import numpy as np
import os 

# Définition d'une nouvelle colonne pour l'analyse de survie

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/ProjetHSB/base_fccss_igr_curie_011021_update_070622_projet_hsb.csv",
                 sep = ",", low_memory = False)
df.head()

Mounted at /content/drive


,personid,num_enq,num_cent,ctr,numcent,cle,ctr2,numcent2,cle2,numero,...,iccc_11.b,iccc_11.c,iccc_11.d,iccc_11.e,iccc_11.f,iccc_12.a,iccc_12.b,iccc_nan,age_at_diagnosis,categ_age_at_diagnosis
0,4141.0,10029085.0,650974,12,650974,NaN,NaN,NaN,NaN,56.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
1,4142.0,10029086.0,671576,12,671576,NaN,NaN,NaN,NaN,57.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0
2,NaN,10029087.0,680197,12,680197,NaN,NaN,NaN,NaN,153.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0.0
3,NaN,10029088.0,681155,12,681155,NaN,NaN,NaN,NaN,60.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0
4,NaN,10029089.0,681460,12,681460,NaN,NaN,NaN,NaN,154.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0


In [ ]:
df['numcent'] == 198114161

NameError: ignored

In [ ]:
df.date_sortie= pd.to_datetime(df.date_sortie, infer_datetime_format=True)
df.datederm = pd.to_datetime(df.datederm, infer_datetime_format=True)
df.date_dvigr = pd.to_datetime(df.date_dvigr, infer_datetime_format=True)
df.date_rep = pd.to_datetime(df.date_rep, infer_datetime_format=True)
df.date_rep2 = pd.to_datetime(df.date_rep2, infer_datetime_format=True)
df.cslt_date_cslt = pd.to_datetime(df.cslt_date_cslt, infer_datetime_format=True)
df.date_diag = pd.to_datetime(df.date_diag, infer_datetime_format=True)
df['other'] = pd.to_datetime(["31/12/2016" for x in range(len(df))], infer_datetime_format=True)


df['date_max'] = df[['date_sortie', 'datederm', 'date_dvigr', 'date_rep', 'date_rep2', 'cslt_date_cslt', 'date_diag', 'other']].max(axis=1)

In [ ]:
df['date_end']= 0

# Si maladie cardiaque observée
card_disease = (df.Pathologie_cardiaque_3_new==1)
df.loc[card_disease, "date_end"] = df[card_disease].date_pathol_cardiaque_3_new


# Si décès et pas de maladie cardiaque => événement censuré
deces_sans_card_notna = ((df.Pathologie_cardiaque_3_new==0)&
                          (df.deces==1)&
                         ~(df["date_deces"].isna()))
deces_sans_card_isna = ((df.Pathologie_cardiaque_3_new==0)&
                        (df.deces==1)&
                        (df["date_deces"].isna()))

df.loc[deces_sans_card_notna, "date_end"] = df[deces_sans_card_notna].date_deces
df.loc[deces_sans_card_isna, "date_end"] = df[deces_sans_card_isna].date_sortie


# Si pas de décès et pas de maladies cardiaque :
no_card_no_death = ((df.deces==0)&(df.Pathologie_cardiaque_3_new==0))

df.loc[no_card_no_death, "date_end"] = df[no_card_no_death].date_max

# Si patient n°199103047
df.loc[(df.numcent==199103047), "date_end"]="03/11/2019"

In [ ]:
df.date_end= pd.to_datetime(df.date_end, infer_datetime_format=True)
df.date_diag = pd.to_datetime(df.date_diag, infer_datetime_format=True)

df['card_age'] = (df.date_end - df.date_diag)/np.timedelta64(1, 'Y')

In [ ]:
# Enregistrement du fichier CSV

new_file_name = "base_fccss_igr_curie_011021_update_260123_projet_hsb.csv"

if new_file_name not in os.listdir("/content/drive/My Drive/Colab Notebooks/ProjetHSB"):
  df.to_csv(f"/content/drive/My Drive/Colab Notebooks/ProjetHSB/{new_file_name}",sep = ",", index=False)
  print("fichier enregistré")

else:
  print("fichier déjà existant")
  col_unnamed = [col for col in df.columns if 'Unnamed' in col]
  if len(col_unnamed ) != 0: 
    for col in col_unnamed: 
      print("Réenregistrement sans les colonnes Unnamed") 
      df=df.drop(columns=col, axis=1)
    df.to_csv(f"/content/drive/My Drive/Colab Notebooks/ProjetHSB{new_file_name}",sep = ",", index=False)
  print("fichier correctement réenregistré")

fichier déjà existant
Réenregistrement sans les colonnes Unnamed
Réenregistrement sans les colonnes Unnamed
fichier correctement réenregistré


# Merge des deux csv

In [ ]:
df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/ProjetHSB/base_fccss_igr_curie_011021_update_260123_projet_hsb.csv",
                 sep = ",", low_memory = False)

In [ ]:
df.head()

,personid,num_enq,num_cent,ctr,numcent,cle,ctr2,numcent2,cle2,numero,...,iccc_11.f,iccc_12.a,iccc_12.b,iccc_nan,age_at_diagnosis,categ_age_at_diagnosis,other,date_max,date_end,card_age
0,4141.0,10029085.0,650974,12,650974,NaN,NaN,NaN,NaN,56.0,...,0.0,0.0,0.0,0.0,0,0.0,2016-12-31,2016-12-31,2016-12-31,51.510982
1,4142.0,10029086.0,671576,12,671576,NaN,NaN,NaN,NaN,57.0,...,0.0,0.0,0.0,0.0,1,0.0,2016-12-31,2020-02-19,2020-02-19,52.346044
2,NaN,10029087.0,680197,12,680197,NaN,NaN,NaN,NaN,153.0,...,0.0,0.0,0.0,0.0,2,0.0,2016-12-31,2020-02-19,2020-02-19,52.072253
3,NaN,10029088.0,681155,12,681155,NaN,NaN,NaN,NaN,60.0,...,0.0,0.0,0.0,0.0,1,0.0,2016-12-31,2020-02-19,2020-02-19,51.645140
4,NaN,10029089.0,681460,12,681460,NaN,NaN,NaN,NaN,154.0,...,0.0,0.0,0.0,0.0,0,0.0,2016-12-31,2020-02-19,2020-02-19,51.519196


In [ ]:
#On importe les deux datasets à merge
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/ProjetHSB/base_fccss_igr_curie_011021_update_110123_projet_hsb.csv",
                 sep = ",", low_memory = False)

print(df.shape)
dose = pd.read_csv("/content/drive/My Drive/Colab Notebooks/ProjetHSB/doses_volumes_patients_update_110123.csv",
                 sep = ",", low_memory = False)
print(dose.shape)

#On les merge sur les colonnes ctr et numcent
#On ne garde que les colonnes Pathologie et card_age, qui sont fusionnés au csv des doses

col = ['Pathologie_cardiaque_3_new', 'card_age', 'ctr', 'numcent', 'do_ALKYL', 'do_ANTHRA','do_VINCA', 'ANTHRA', 'ALKYL', 'VINCA']
new_df = pd.merge(dose, df.loc[:, col], on= ['ctr', 'numcent'])
new_df.head()
print(new_df.shape)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(7670, 15774)
(3943, 148)
(3943, 156)


In [ ]:
new_df.to_csv(f"/content/drive/My Drive/Colab Notebooks/ProjetHSB/doses_volumes_patients_final_update_110123.csv",sep = ",", index=False)

In [ ]:
# Enregistrement du fichier CSV
df_to_save = new_df
new_file_name = "doses_volumes_patients_update_110123.csv"

if new_file_name not in os.listdir("/content/drive/My Drive/Colab Notebooks/ProjetHSB"):
  df.to_csv(f"/content/drive/My Drive/Colab Notebooks/ProjetHSB/{new_file_name}",sep = ",", index=False)
  print("fichier enregistré")

else:
  print("fichier déjà existant")
  df_to_save = pd.read_csv(f"/content/drive/My Drive/Colab Notebooks/ProjetHSB/{new_file_name}",
                 sep = ",", low_memory = False)
  col_unnamed = [col for col in df_to_save.columns if 'Unnamed' in col]
  print(col_unnamed)
  if len(col_unnamed ) != 0: 
    for col in col_unnamed: 
      print("Réenregistrement sans les colonnes Unnamed") 
      df_to_save = df_to_save.drop(columns=col, axis=1)
    df_to_save.to_csv(f"/content/drive/My Drive/Colab Notebooks/ProjetHSB/{new_file_name}",sep = ",", index=False)
    print("fichier correctement réenregistré")

fichier déjà existant
[]


In [ ]:
new_file_name = "doses_volumes_patients_update_110123.csv"
new_df.to_csv(f"/content/drive/My Drive/{new_file_name}",sep = ",", index=False)
new_df

,ctr,numcent,dv_V01_1320,dv_V05_1320,dv_V1_1320,dv_V2_1320,dv_V5_1320,dv_V10_1320,dv_V15_1320,dv_V20_1320,...,dv_D95_324,dv_D99_324,Pathologie_cardiaque_3_new,card_age,do_ALKYL,do_ANTHRA,do_VINCA,ANTHRA,ALKYL,VINCA
0,3,197204357,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.00000,0.00000,0,47.554707,0.000000,0.000000,0.000000,0,0,0
1,3,197608259,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.03700,0.03600,1,20.545254,98731.021583,575.179856,27.431655,1,1,1
2,3,197704050,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.00000,0.00000,0,42.681232,0.000000,0.000000,0.000000,0,0,0
3,3,197704181,100.0,44.532788,12.736392,0.000000,0.000000,0.000000,0.000000,0.0,...,0.23200,0.22100,0,42.818128,4924.440694,270.202507,60.120058,1,1,1
4,3,197704641,100.0,100.000000,94.521714,22.741981,0.017031,0.000000,0.000000,0.0,...,0.98800,0.93878,0,42.662067,0.000000,0.000000,0.000000,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3938,8,7607979,100.0,2.580092,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.21200,0.20200,0,43.346544,2476.190476,49.523810,4.952381,1,1,1
3939,8,7702001,100.0,13.617021,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.30000,0.28900,0,10.154897,0.000000,0.000000,0.000000,0,0,0
3940,8,7709338,100.0,100.000000,84.606813,0.000000,0.000000,0.000000,0.000000,0.0,...,0.76900,0.74100,0,42.180195,0.000000,0.000000,0.000000,0,0,0
3941,8,7901491,100.0,100.000000,100.000000,100.000000,99.351659,93.524171,43.969668,0.0,...,8.73300,6.23800,0,40.994682,14240.316832,0.000000,19.103960,0,1,1


In [ ]:
dose = pd.read_csv("/content/drive/My Drive/Colab Notebooks/ProjetHSB/doses_volumes_patients_update_110123.csv",
                 sep = ",", low_memory = False)
dose.head()

,ctr,numcent,dv_V01_1320,dv_V05_1320,dv_V1_1320,dv_V2_1320,dv_V5_1320,dv_V10_1320,dv_V15_1320,dv_V20_1320,...,dv_D40_324,dv_D50_324,dv_D60_324,dv_D70_324,dv_D80_324,dv_D90_324,dv_D95_324,dv_D99_324,Pathologie_cardiaque_3_new,card_age
0,3,197204357,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.000,0.000,0.000,0.0000,0.0000,0.000,0.000,0.00000,0,47.554707
1,3,197608259,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.044,0.043,0.042,0.0410,0.0390,0.037,0.037,0.03600,1,20.545254
2,3,197704050,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.000,0.000,0.000,0.0000,0.0000,0.000,0.000,0.00000,0,42.681232
3,3,197704181,100.0,44.532788,12.736392,0.000000,0.000000,0.0,0.0,0.0,...,0.462,0.355,0.313,0.2830,0.2590,0.240,0.232,0.22100,0,42.818128
4,3,197704641,100.0,100.000000,94.521714,22.741981,0.017031,0.0,0.0,0.0,...,1.562,1.407,1.336,1.2197,1.1538,1.027,0.988,0.93878,0,42.662067
